In [1]:
# -*- coding: utf-8 -*-
import requests
import re
from bs4 import BeautifulSoup as BS
import pandas as pd
import json
from collections import Counter
from urllib.error import HTTPError
import numpy as np
from tqdm import trange
from http.client import IncompleteRead

In [4]:
All_data = pd.read_excel('./Alldata.xlsx',keep_default_na=False).drop(columns=['Unnamed: 0'],axis=1)
indices = []
for i in range(len(All_data)):
    line = All_data.iloc[i]
    indices.append((line['t'],regex(line['area'])))
All_data['index'] = indices
All_data['gov_report'] = ''
All_data['stat_report'] = ''

In [3]:
def regex(name):
    if name == "和田地区和田县" or name == "和田地区和田市":
        return name[-3:]
    
    name = name.strip()
    name = name.strip('\t')
    special = ['城区','矿区','郊区'] ## with which patterns can only use exact match
    for item in special:
        if item in name:
            return name
    
    with open('./ethicity.json','r') as f:
        ethnicities = json.loads(f.read())
    ethnicity = [item["name"] for item in ethnicities] + ["各族"]

    for eth in ethnicity:
        if eth in name:
            name = name.replace(eth,'')
            
    if '地区' in name:
        name = [item for item in name.split("地区") if len(item)>0][-1]
    if '市' in name:
        name = [item for item in name.split("市") if len(item)>0][-1]
    elif '自治州' in name:
        name = [item for item in name.split("自治州") if len(item)>0][-1]
    
    if len(name) > 2:
        name = name.strip('区')
        name = name.strip('旗')
        name = name.strip('自治县')
        name = name.strip('省')
    
    if len(name) > 2:
        name = name.strip('县')
        
    return name

(2013, '卢氏')

In [10]:
## crawl data in areas listed in indices
def craw_and_write(web_path_root, to_be_matched_list = indices):
    global All_data
    if "tongjigongbao" in web_path_root:
        entry = 'stat_report'
    else:
        entry = 'gov_report'
    count_dict = {key:0 for key in to_be_matched_list}
    
    for i in trange(25000):
        title, body = craw_single(f'{web_path_root}/{i}.html')
        ## try matching
        if title:
            for k in range(len(to_be_matched_list)): ## item: (t, area)
                item = to_be_matched_list[k]
                if (str(item[0]) in title) and (item[1] in title): ## if matched
                    print(item)
                    All_data.loc[k,entry] = (title+'\t'+body) ## data: (title, body)
                    count_dict[item] += 1
                    break
        ## whether matched successfully, go to next
        i += 1
    return count_dict


In [17]:
def craw_single(web_path):
    r = requests.get(web_path).text.encode('ISO-8859-1').decode('gbk')
    title = re.findall(r'<div class="title"><h1>(.*?)</h1></div>',r)
    if len(title) > 0: ## if not "404 Not Found" returned
        title = title[0]
        d = pd.read_html(web_path,flavor='bs4')
        body = d[0][0].values[np.argmax([len(item) for item in d[0][0].values])]
        j = 2
        while True:
            try:
                e = pd.read_html(web_path[:-5]+f'_{j}'+'.html',flavor='bs4')
                if len(e)>0:
                    data_ = e[0][0].values[np.argmax([len(item) for item in e[0][0].values])]
                    body = body + data_
                j += 1
            except:
                return (title, body)
    else:
        return None, None
            
    

In [ ]:
## main ##
count_dict = craw_and_write('https://www.ahmhxc.com/tongjigongbao/')
All_data.to_excel('./WITH_STAT_REPORT.xlsx')
with open('./stat_report.json','w') as g:
    json.dump(count_dict)

KeyboardInterrupt: 

In [18]:
craw_single('https://www.ahmhxc.com/gongzuobaogao/5025.html')

('（江苏省）2016年常州市天宁区人民政府工作报告（全文）',
 '——2016年1月6日在常州市天宁区第十五届人民代表大会第五次会议上 \t代区长 许小波 \t各位代表： \t现在，我代表常州市天宁区人民政府，向大会作工作报告，请予审议，并请各位政协委员和列席人员提出意见。 \t2015年工作和“十二五”发展回顾 \t2015年是冲刺“十二五”的决战之年。一年来，面对严峻复杂的国内外经济形势和艰巨繁重的改革发展任务，在市委、市政府和区委的正确领导下，全区上下主动适应经济发展新常态，攻坚克难，砥砺奋进，统筹做好稳增长、调结构、促改革、惠民生等各项工作，较好地完成了区十五届人大四次会议确定的各项目标任务。 \t区域经济平稳增长。一年来，我们加大有效投入，深入开展“重大项目深化年”活动，经济总体呈现稳中有进、进中提质的发展态势。综合实力稳健提升。预计，全年完成地区生产总值635亿元，增长8%；全社会固定资产投资360亿元，增长5.5%；公共财政预算收入44.8亿元，增长6%；社会消费品零售总额430亿元，增长10.8%。项目建设有力推进。全区97个重点项目累计完成投资115亿元，大娘水饺冷链物流、强力先端固化材料等6个市重点新建项目全面开工，亚玛顿光电玻璃（四期）、四药厂研发中心（二期）等4个市重点续建项目进展顺利，全年成功引进总投资超千万美元外资项目3个。企业发展亮点纷呈。常州四药新建3万平方米高标准厂房，为后续加速发展奠定了扎实基础；艾贝服饰全国开设专卖店超100家，自主品牌建设实现突破；江海环保与美国艺康达成战略合作，股权交易额达1.69亿美元；华利达、东奥进出口总额保持两位数增长，分别达到3.6亿美元、2.5亿美元；亚玛顿在迪拜投资办厂，迈开“走出去”步伐；大洋线缆、中航蜂窝等7家企业“新三板”成功挂牌，恒丰铜材、华东石油等7家企业完成股改。 \t转型升级步伐加快。一年来，我们坚定转型方向，强化科技创新，助推产业升级。三次产业调优调强。常州市健康养老服务业集聚区获批省级现代服务业集聚区，预计，全区服务业年营业收入突破2000亿元，达2120亿元、增长15.2%，服务业增加值432亿元、增长9.96%。完成工业技改投入73.1亿元，全年实现规上工业总产值1097亿元、增长7.2%。成功申报部级水稻高产增效创建万亩示范片项目，双丰水产养殖专业合作社创成市级现代农业

In [17]:
'''
Author: Xiaocong Yang
LastEditors: Xiaocong Yang
'''
# -*- coding: utf-8 -*-
from typing import final
import requests
import re
from bs4 import BeautifulSoup as BS
import pandas as pd
import json
from collections import Counter
from urllib.error import HTTPError
import numpy as np
from tqdm import trange
import argparse


def regex(name):
    if name == "和田地区和田县" or name == "和田地区和田市":
        return name[-3:]
    
    name = name.strip()
    name = name.strip('\t')
    special = ['城区','矿区','郊区'] ## with which patterns can only use exact match
    for item in special:
        if item in name:
            return name
    
    with open('./ethicity.json','r') as f:
        ethnicities = json.loads(f.read())
    ethnicity = [item["name"] for item in ethnicities] + ["各族"]

    for eth in ethnicity:
        if eth in name:
            name = name.replace(eth,'')
    print(name)
            
    if '地区' in name:
        name = [item for item in name.split("地区") if len(item)>0][-1]
    if '市' in name:
        name = [item for item in name.split("市") if len(item)>0][-1]
    if '自治州' in name:
        name = [item for item in name.split("自治州") if len(item)>0][-1]
    
    if len(name) > 2:
        name = name.strip('区')
        name = name.strip('旗')
        name = name.strip('自治县')
        name = name.strip('省')
    
    if len(name) > 2:
        name = name.strip('县')
        
    return name

## crawl data in areas listed in indices
def craw_and_write(web_path_root, to_be_matched_list, start, end):
    global All_data
    if "tongjigongbao" in web_path_root:
        entry = 'stat_report'
    else:
        entry = 'gov_report'
    count_dict = {str(key):0 for key in to_be_matched_list}
    
    ## main loop
    for i in trange(start, end, 1):
        ## try to catch any Exception and continue to ensure a finally outcome
        try:
            web_path = f'{web_path_root}/{i}.html'
            r = requests.get(web_path).text.encode('ISO-8859-1').decode('gbk')
            title = re.findall(r'<div class="title"><h1>(.*?)</h1></div>',r)
            if len(title) > 0:  ## if not "404 Not Found" returned
                title = title[0]
                ## try matching title first. if matched then call for body content
                for k in range(len(to_be_matched_list)): ## item: (t, area)
                    item = to_be_matched_list[k]
                    if (str(item[0]) in title) and (item[1] in title) and item[1] != '河南': ## if matched
                        print('ckpt1')
                        body = craw_single(web_path)
                        print(body)
                        All_data.loc[k,entry] = (title+'\t'+body) ## data: (title, body)
                        count_dict[str(item)] += 1
                        break
        except:
            pass
            
    return count_dict

def craw_single(web_path):
    d = pd.read_html(web_path,flavor='bs4')
    body = d[0].iloc[:,0].values[np.argmax([len(str(item)) for item in d[0].iloc[:,0].values])]
    j = 2
    while True:
        try:
            e = pd.read_html(web_path[:-5]+f'_{j}'+'.html',flavor='bs4')
            if len(e)>0:
                data_ = e[0].iloc[:,0].values[np.argmax([len(item) for item in e[0].iloc[:,0].values])]
                body = body + data_
            j += 1
        except:
            return body




In [19]:
regex('琼中黎族苗族自治县琼中黎族苗族自治县')

琼中自治县琼中自治县


'琼中自治县琼中'

In [41]:
All_data = pd.read_excel('./Alldata.xlsx',keep_default_na=False).drop(columns=['Unnamed: 0'],axis=1)
indices = []
for i in range(len(All_data)):
    line = All_data.iloc[i]
    indices.append((line['t'],regex(line['area'])))
All_data['index'] = indices
All_data['gov_report'] = ''
All_data['stat_report'] = ''
count_dict = craw_and_write('https://www.ahmhxc.com/tongjigongbao/',indices, 18974, 18977)



三门峡市卢氏县
三门峡市卢氏县
三门峡市卢氏县
三门峡市卢氏县
三门峡市卢氏县
三门峡市卢氏县
三门峡市卢氏县
上饶市上饶县
上饶市上饶县
上饶市上饶县
上饶市上饶县
上饶市上饶县
上饶市上饶县
上饶市上饶县
上饶市余干县
上饶市余干县
上饶市余干县
上饶市余干县
上饶市余干县
上饶市余干县
上饶市余干县
上饶市横峰县
上饶市横峰县
上饶市横峰县
上饶市横峰县
上饶市横峰县
上饶市横峰县
上饶市横峰县
上饶市鄱阳县
上饶市鄱阳县
上饶市鄱阳县
上饶市鄱阳县
上饶市鄱阳县
上饶市鄱阳县
上饶市鄱阳县
中卫市海原县
中卫市海原县
中卫市海原县
中卫市海原县
中卫市海原县
中卫市海原县
中卫市海原县
临夏自治州东乡县
临夏自治州东乡县
临夏自治州东乡县
临夏自治州东乡县
临夏自治州东乡县
临夏自治州东乡县
临夏自治州东乡县
临夏自治州临夏县
临夏自治州临夏县
临夏自治州临夏县
临夏自治州临夏县
临夏自治州临夏县
临夏自治州临夏县
临夏自治州临夏县
临夏自治州临夏市
临夏自治州临夏市
临夏自治州临夏市
临夏自治州临夏市
临夏自治州临夏市
临夏自治州临夏市
临夏自治州临夏市
临夏自治州和政县
临夏自治州和政县
临夏自治州和政县
临夏自治州和政县
临夏自治州和政县
临夏自治州和政县
临夏自治州和政县
临夏自治州广河县
临夏自治州广河县
临夏自治州广河县
临夏自治州广河县
临夏自治州广河县
临夏自治州广河县
临夏自治州广河县
临夏自治州康乐县
临夏自治州康乐县
临夏自治州康乐县
临夏自治州康乐县
临夏自治州康乐县
临夏自治州康乐县
临夏自治州康乐县
临夏自治州永靖县
临夏自治州永靖县
临夏自治州永靖县
临夏自治州永靖县
临夏自治州永靖县
临夏自治州永靖县
临夏自治州永靖县
临夏自治州积石山县
临夏自治州积石山县
临夏自治州积石山县
临夏自治州积石山县
临夏自治州积石山县
临夏自治州积石山县
临夏自治州积石山县
临汾市吉县
临汾市吉县
临汾市吉县
临汾市吉县
临汾市吉县
临汾市吉县
临汾市吉县
临汾市大宁县
临汾市大宁县
临汾市大宁县
临汾市大宁县
临汾市大宁县
临汾市大宁县
临汾市大宁县
临汾市永和县
临汾市永和县
临汾市永和县
临汾市永和县
临汾市永和县
临汾市永和县
临汾市永和县
临汾市汾西县
临汾市汾西县
临汾市汾西县
临汾市汾西县
临汾市汾西县
临汾市汾西县
临汾市汾西县

  0%|          | 0/3 [00:00<?, ?it/s]


阿克苏地区乌什县
阿克苏地区乌什县
阿克苏地区乌什县
阿克苏地区乌什县
阿克苏地区乌什县
阿克苏地区乌什县
阿克苏地区乌什县
阿克苏地区柯坪县
阿克苏地区柯坪县
阿克苏地区柯坪县
阿克苏地区柯坪县
阿克苏地区柯坪县
阿克苏地区柯坪县
阿克苏地区柯坪县
阿勒泰地区吉木乃县
阿勒泰地区吉木乃县
阿勒泰地区吉木乃县
阿勒泰地区吉木乃县
阿勒泰地区吉木乃县
阿勒泰地区吉木乃县
阿勒泰地区吉木乃县
阿勒泰地区青河县
阿勒泰地区青河县
阿勒泰地区青河县
阿勒泰地区青河县
阿勒泰地区青河县
阿勒泰地区青河县
阿勒泰地区青河县
阿坝自治州九寨沟县
阿坝自治州九寨沟县
阿坝自治州九寨沟县
阿坝自治州九寨沟县
阿坝自治州九寨沟县
阿坝自治州九寨沟县
阿坝自治州九寨沟县
阿坝自治州壤塘县
阿坝自治州壤塘县
阿坝自治州壤塘县
阿坝自治州壤塘县
阿坝自治州壤塘县
阿坝自治州壤塘县
阿坝自治州壤塘县
阿坝自治州小金县
阿坝自治州小金县
阿坝自治州小金县
阿坝自治州小金县
阿坝自治州小金县
阿坝自治州小金县
阿坝自治州小金县
阿坝自治州松潘县
阿坝自治州松潘县
阿坝自治州松潘县
阿坝自治州松潘县
阿坝自治州松潘县
阿坝自治州松潘县
阿坝自治州松潘县
阿坝自治州汶川县
阿坝自治州汶川县
阿坝自治州汶川县
阿坝自治州汶川县
阿坝自治州汶川县
阿坝自治州汶川县
阿坝自治州汶川县
阿坝自治州理县
阿坝自治州理县
阿坝自治州理县
阿坝自治州理县
阿坝自治州理县
阿坝自治州理县
阿坝自治州理县
阿坝自治州红原县
阿坝自治州红原县
阿坝自治州红原县
阿坝自治州红原县
阿坝自治州红原县
阿坝自治州红原县
阿坝自治州红原县
阿坝自治州若尔盖县
阿坝自治州若尔盖县
阿坝自治州若尔盖县
阿坝自治州若尔盖县
阿坝自治州若尔盖县
阿坝自治州若尔盖县
阿坝自治州若尔盖县
阿坝自治州茂县
阿坝自治州茂县
阿坝自治州茂县
阿坝自治州茂县
阿坝自治州茂县
阿坝自治州茂县
阿坝自治州茂县
阿坝自治州金川县
阿坝自治州金川县
阿坝自治州金川县
阿坝自治州金川县
阿坝自治州金川县
阿坝自治州金川县
阿坝自治州金川县
阿坝自治州阿坝县
阿坝自治州阿坝县
阿坝自治州阿坝县
阿坝自治州阿坝县
阿坝自治州阿坝县
阿坝自治州阿坝县
阿坝自治州阿坝县
阿坝自治州马尔康市
阿坝自治州马尔康市
阿坝自治州马尔康市
阿坝自治州马尔康市
阿坝自治州马尔

 33%|███▎      | 1/3 [00:12<00:24, 12.44s/it]

ckpt1


In [40]:
All_data['index'][0]

"(2013, '卢氏')"

In [12]:
All_data = pd.read_csv('./outputs/WITH_GOV_REPORT.csv',keep_default_na=False).drop(columns=['Unnamed: 0'],axis=1)
indices = All_data['index']

In [13]:
indices[0]

"(2013, '卢氏')"

In [11]:
indices[0].split(',')[1][:-2]

" '卢氏"

In [ ]:
'''
政府工作报告中可以查到：
1.城镇居民人均可支配收入
2.农村居民人均可支配收入
3.地区生产总值
4.固定资产投资
5.社会销售品零售总额

国民经济和社会发展统计公报
1.年末总人口（户籍）
2.财政支出
3.普通小学在校生
4. 医院、卫生院床位数
5.农村用电量
'''

In [ ]:
len(d.split(' ')[1])

0

In [ ]:
d = pd.read_html('https://www.ahmhxc.com/tongjigongbao/21204.html',flavor='bs4')[0][0][0]


In [ ]:
d.split(' ')

['湘潭县2020年国民经济和社会发展统计公报',
 '',
 '2020年是极不平凡的一年。面对国内外形势的深刻复杂变化和新冠肺炎疫情的冲击，全县上下在县委、县政府的坚强领导下，坚持以习近平新时代中国特色社会主义思想为指导，认真贯彻习近平总书记考察湖南重要讲话指示精神，全面落实党中央、国务院、省委、省政府和市委、市政府各项决策部署，坚持稳中求进工作总基调，统筹疫情防控和经济社会发展，扎实做好“六稳”工作，全面落实“六保”任务，继续实施“强工惠农”发展战略，开展“五大竞赛”，着力推动县域高质量发展。2020年，全县经济运行呈增速稳步回升、结构持续优化、质效不断改善的态势。',
 '',
 '一、综合',
 '',
 '初步核算，全年实现地区生产总值501.0亿元，增长3.7%。其中，第一产业增加值68.6亿元，增长3.7%;第二产业增加值268.9亿元，增长4.4%;第三产业增加值163.5亿元，增长2.2%。三次产业结构比为13.7:53.7:32.6。',
 '',
 '二、农业',
 '',
 '全年实现农林牧渔业总产值119.5亿元，增长3.8%。全县粮食播种面积83.6千公顷，下降2.9%，粮食总产量60.65万吨，下降2.7%;蔬菜播种面积20.6千公顷，增长3.5%，蔬菜产量64.41万吨，增长4.5%;油料播种面积13.1千公顷，增长18.4%，油料产量2.2万吨，增长18.4%。全年猪牛羊禽肉产量7.2万吨，下降3.9%。其中，猪肉产量7万吨，下降4.3%。禽蛋产量3.0万吨，增长20.3%。水产品产量4.3万吨，增长12.1%。年末出栏生猪96.9万头，下降3.8%;出栏牛1.2万头，增长35.3%;出栏羊6.5万头，增长20.6%。',
 '',
 '全年水利工程投入资金4.6亿元，开工各类水利工程7500处，完成水利工程土方石230万立方米。新增农田有效灌溉面积500公顷，新增节水灌溉面积660公顷。全县年末农业机械总动力达147.1万千瓦。',
 '',
 '三、工业和建筑业',
 '',
 '全年完成规模工业增加值203.4亿元，增长3.4%，其中天易示范区完成规模工业增加值166.7亿元，增长3.8%，占全县规模工业增加值的比重为82.0%。全县规模工业企业实现主营业务收入621.8亿元，下降1.3%，实现利润总额22.8亿元，增长0.